In [1]:
import psycopg2 as psy
from config import config

In [29]:
db = 'idtest2'
table = 'dummy21'

In [30]:
def connectTest():
    

    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psy.connect(**config()) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psy.DatabaseError, Exception) as error:
        print(error)

connectTest()

Connected to the PostgreSQL server.


<connection object at 0x7fb8c4859e40; dsn: 'user=postgres password=xxx host=localhost port=5430', closed: 0>

In [33]:
def connect_Create_db():
    conn = None
    try:
        conn = psy.connect(**config())
        conn.autocommit = True
        print("Connected to the PostgreSQL server")

        crsr = conn.cursor()
        crsr.execute('SELECT version()')
        db_version = crsr.fetchone()
        #crsr.close()
        print(f"PostgreSQL database version: {db_version}")

        dbname = db
        #crsr.execute('CREATE DATABASE pycreatetest ;')
        #crsr.execute('SELECT datname FROM pg_database WHERE datistemplate = false;')
        crsr.execute(f"SELECT datname FROM pg_database WHERE datname = '{dbname}'")
        tables = crsr.fetchall()
        #print(len(tables))
        if len(tables) == 0:
            crsr.execute(f'CREATE DATABASE {dbname}')
            print('DB created')

        else:
            print('DB exists')
        crsr.close()

    except(Exception, psy.DatabaseError) as error:
        print(error)

    finally:
        if conn != None:
            conn.close()
            print("Connection closed")

connect_Create_db()

Connected to the PostgreSQL server
PostgreSQL database version: ('PostgreSQL 16.6 (Debian 16.6-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
DB exists
Connection closed


In [34]:
def create_table():
    '''
    Create postgres table to store images and data
    '''
    try:
        conn = psy.connect(**config(),database=db) #Hardcoded database choice, can be moved to ini later
        conn.autocommit = True
        crsr = conn.cursor()

        crsr.execute(f'''SELECT EXISTS ( SELECT FROM information_schema.tables WHERE table_name = '{table}');''')
        
        exists = bool(crsr.fetchone()[0])
        print(exists)
        if (exists == True):
            print('Table exists')
        else:
            crsr.execute(f'''CREATE TABLE {table} (id INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY, image TEXT, label INT, prediction INT, correct BOOL, certainty REAL);''')
            print('Table created')
        
    except(Exception, psy.DatabaseError) as error:
        print (error)

    finally:
        crsr.close()
        conn.close()

create_table()

True
Table exists
